In [7]:
# Author: Davide Aloi - PhD student - University of Birmingham
# Description: the script correlates current density values within M1 and Th for each 
# participant with their respective DCM values.

# Imports
import warnings
warnings.filterwarnings("ignore")
import os
import glob
import numpy as np
from nilearn import image, plotting
from nilearn.image import new_img_like
from scipy import ndimage
import scipy.io
import time
from tqdm import tqdm

In [8]:
## Parameters and variables: 
results_folder = 'D:\\roast-chapter3\\wp_all_results\\' # Folder with results
main_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\' # Project folder

# Datasets names and subjects lists
db_names = ['wp2a', 'wp2a', 'wp1a', 'wp1b']
# dataset names for the dcm results (pairwise int)
db_names_dcm = ['wp2a_day1_pairwise','wp2a_day5_pairwise', 'wp1a_pairwise', 'wp1b_pairwise']
dcm_results_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast\\all_dcm_results\\' # Folder with dcm results

db_subjects = [['01','02','03','04','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','22','23','24'], # Wp2a
               ['03','04','05','07','09','10','11','12','13','15','16','17','18','19','20','21','22','23','24','25','26'], # Wp1a
               ['01','02','03','04','05','06','07','08','09','10','11','12','13','15','16','17','18','19','21','22','23']] # Wp1b                              

## Loading AAL3 atlas and extracting M1 / Thalamus ROIs (regions of interest)
# AAL3 atlas paper: https://www.oxcns.org/papers/607%20Rolls%20Huang%20Lin%20Feng%20Joliot%202020%20AAL3.pdf 
AAl3_path = os.path.join(main_folder, 'rois', 'AAL3v1_1mm.nii')
AAL3_atlas = image.load_img(AAl3_path)

## Creating M1, Th and cerebellar masks from the AAL3 atlas. Load MNI template.
# AAL3 index for left M1 = 1
m1 = image.math_img("np.where(img == 1, 1, 0)", img = AAL3_atlas) 
# AAL3 index for TH = 121 - 149 (odd values only (left thalamus)) --> I'm not convinced about this one, ask Davinia
th = image.math_img("np.where(np.isin(img, np.arange(121, 150, 2)), 1, 0)", img = AAL3_atlas) 
# AAL3 index for right Cerebellum (cb) (102 and 108: cerebellar lobes IV-V and VIII)
cb = image.math_img("np.where(np.isin(img, np.array([102, 108])), 1, 0)", img = AAL3_atlas)

In [9]:
## Iterate all three datasets
from scipy import stats
from statsmodels.stats import multitest

for db_id, db in tqdm(enumerate(db_names)):
    ## Loading results for each dataset (current density maps)
    cd_maps = image.load_img(os.path.join(results_folder, db + '_all_cd_maps.nii'))
    print('\nAnalysing dataset ' + db + ' - N. subjects: ' + str(cd_maps.shape[3]))
    print('Current density map shape: ', cd_maps.shape)

    ## Loading DCM results (nb: the unthresholded ones, for the correlation analysis)
    DCM = np.load(os.path.join(dcm_results_folder, db_names_dcm[db_id] + '_dcm_unthresholded.npy'))
    print('DCM array shape: ', DCM.shape)

    ## Conversion of diagonal values to Hz + recentering on 0
    funcHz = lambda x: (-0.5*(np.exp(x))) if x != 0 else 0

    # Positive values now indicate less self inhibition
    # Negative values indicate more self inhibition
    for i in range(0, DCM.shape[0]): 
        diag = np.diagonal(DCM[i,:,:])
        diag_converted = []
        for element in diag:
            diag_converted.append(funcHz(element))
        np.fill_diagonal(DCM[i,:,:], np.array(diag_converted))
    
    ## Resampling masks
    m1_resampled = image.resample_to_img(m1, cd_maps, interpolation = 'nearest').get_fdata()
    th_resampled = image.resample_to_img(th, cd_maps, interpolation = 'nearest').get_fdata()
    cb_resampled = image.resample_to_img(cb, cd_maps, interpolation = 'nearest').get_fdata()
    
    # List of roise and connections 
    rois = [['m1', m1_resampled],
            ['th', th_resampled],
            ['cb', cb_resampled]]

    cons = [['m1m1', DCM[:,0,0]],
            ['m1th', DCM[:,1,0]],
            ['thth', DCM[:,0,1]],
            ['thm1', DCM[:,1,1]]]

    cd_maps_a = cd_maps.get_fdata() # current density data in numpy format (easier to access)
    shape = cd_maps.shape
    df = shape[3] # degrees of freedom (N participants)

    for roi_name, roi in rois:
         print('Checking roi: ', roi_name)
         for con_name, con in cons:
            print('Correlating with connection: ', con_name)      
            # arrays where we'll save our results (1 per con/roi/db)
            all_p = [] # list of p values for subsequent correction 
            # maps where we'll save the results (not used if no voxels survive....)
            cormap_r = np.zeros(m1_resampled.shape)
            cormap_t = np.zeros(m1_resampled.shape)
            cormap_p = np.zeros(m1_resampled.shape)

            # let's now iterate each voxel in the roi (but consider only those == 1)
            for i, v in enumerate(roi.flatten()):
                if v != 0: # if voxel is not masked 
                    # trick to extract the 3d coordinates
                    idx, idy, idz = np.unravel_index(i,(shape[0],shape[1],shape[2]))
                    p_v = cd_maps_a[idx,idy,idz,:] # value for that voxel for all participants
                    cormap_r[idx,idy,idz], cormap_p[idx,idy,idz] = stats.pearsonr(p_v, con) # calculate r, p
                    cormap_t[idx,idy,idz] = scipy.stats.t.ppf(q=1-cormap_p[idx,idy,idz], df=df) # find t
                    all_p.append(cormap_p[idx,idy,idz])
            
            # Printing results 
            print('Number of significant voxels (unc): ',((cormap_p < 0.05) & (cormap_p > 0)).sum())
            pcor = multitest.multipletests(all_p, alpha=0.05, method = 'fdr_bh')
            print('Number of significant voxels (fdr): ', (pcor[1] < 0.05).sum())

            # if there are significant voxels after the correction, save all maps
            if (pcor[1] < 0.05).sum() > 0: 
                cormap_p_cor = roi.flatten()
                idx_pcor = 0
                for idx_cpc, cpc in enumerate(cormap_p_cor):
                    if cpc == 1:
                        cormap_p_cor[idx_pcor] = pcor[1][idx_pcor]
                        idx_pcor+=1
                cormap_p_cor = cormap_p_cor.reshape(roi.shape)
                # save corrected map and r 
                new_img_like(cd_maps, cormap_r).to_filename(db + '_' + roi_name + '_' + con_name + '_r.nii')
                new_img_like(cd_maps, cormap_p_cor).to_filename(db + '_' + roi_name + '_' + con_name + '_p_cor.nii')



    ## Applying masks (this is actually a stupid step, it's not needed)
    #M1_all = cd_maps.get_fdata() * np.repeat(m1_resampled[:, :, :, np.newaxis], cd_maps.shape[3], axis=3)
    #TH_all = cd_maps.get_fdata() * np.repeat(th_resampled[:, :, :, np.newaxis], cd_maps.shape[3], axis=3)
    #CB_all = cd_maps.get_fdata() * np.repeat(cb_resampled[:, :, :, np.newaxis], cd_maps.shape[3], axis=3) 


0it [00:00, ?it/s]


Analysing dataset wp2a - N. subjects: 22
Current density map shape:  (157, 189, 156, 22)
DCM array shape:  (22, 4, 4)
Checking roi:  m1
Correlating with connection:  m1m1
Number of significant voxels (unc):  542
Number of significant voxels (fdr):  0
Correlating with connection:  m1th
Number of significant voxels (unc):  585
Number of significant voxels (fdr):  0
Correlating with connection:  thth
Number of significant voxels (unc):  2121
Number of significant voxels (fdr):  0
Correlating with connection:  thm1
Number of significant voxels (unc):  1156
Number of significant voxels (fdr):  0
Checking roi:  th
Correlating with connection:  m1m1
Number of significant voxels (unc):  5828
Number of significant voxels (fdr):  5530
Correlating with connection:  m1th
Number of significant voxels (unc):  12
Number of significant voxels (fdr):  0
Correlating with connection:  thth
Number of significant voxels (unc):  3473
Number of significant voxels (fdr):  0
Correlating with connection:  thm1

1it [01:11, 71.33s/it]

Number of significant voxels (unc):  2318
Number of significant voxels (fdr):  0

Analysing dataset wp2a - N. subjects: 22
Current density map shape:  (157, 189, 156, 22)
DCM array shape:  (22, 4, 4)
Checking roi:  m1
Correlating with connection:  m1m1
Number of significant voxels (unc):  2874
Number of significant voxels (fdr):  0
Correlating with connection:  m1th
Number of significant voxels (unc):  436
Number of significant voxels (fdr):  0
Correlating with connection:  thth
Number of significant voxels (unc):  2527
Number of significant voxels (fdr):  0
Correlating with connection:  thm1
Number of significant voxels (unc):  595
Number of significant voxels (fdr):  0
Checking roi:  th
Correlating with connection:  m1m1
Number of significant voxels (unc):  242
Number of significant voxels (fdr):  0
Correlating with connection:  m1th


1it [01:51, 111.63s/it]


KeyboardInterrupt: 

In [31]:
pcor = multitest.multipletests(all_p, alpha=0.05, method = 'fdr_bh')

cormap_p_cor = roi.flatten()
id = 0
for idx_cpc, cpc in enumerate(cormap_p_cor):
    if cormap_p_cor[idx_cpc] == 1:
        cormap_p_cor[idx_cpc] == pcor[1][id]
        id += 1

IndexError: index 2531 is out of bounds for axis 0 with size 2531